OCR utilizanod SVM y libsvm
------------------------------
Extraido del libro "Programing computer vision with python"
Link: http://programmingcomputervision.com/downloads/ProgrammingComputerVision_CCdraft.pdf#page=223&zoom=100,97,325

In [ ]:
import numpy as np
from PIL import Image
from libsvm.svmutil import *
import os

In [ ]:
## Paths
path_train = r"..\..\Data\dataset\text_arial"
path_test = r"..\..\Data\dataset\text_arial_test"

In [ ]:
## Dataset Loader
def load_ocr_data(path, limit_per_class=None):
    imlist = []
    counters = {}  # contador de imágenes por subcarpeta/clase

    for root, dirs, files in os.walk(path):
        # obtenemos el nombre de la carpeta padre como clase
        class_name = os.path.basename(root)
        if not class_name.isdigit():
            continue  # ignoramos carpetas que no son números

        # inicializamos contador si no existe
        if class_name not in counters:
            counters[class_name] = 0

        for f in files:
            if f.lower().endswith('.png'):
                if limit_per_class is not None and counters[class_name] >= limit_per_class:
                    continue  # ya llegamos al límite en esta clase
                imlist.append(os.path.join(root, f))
                counters[class_name] += 1

    labels = [int(os.path.basename(imfile).split('_')[0]) for imfile in imlist]

    feature = []
    for image in imlist:
        im = np.array(Image.open(image).convert('L'),dtype=float) / 255
        feature.append(im.flatten())

    return np.array(feature), labels

In [ ]:
## Traning data
feature, labels = load_ocr_data(path_train,100)

## Test data
feature_test, labels_test = load_ocr_data(path_test,100)


import random

combined = list(zip(feature, labels))
random.shuffle(combined)
feature[:], labels[:] = zip(*combined)

combined = list(zip(feature_test, labels_test))
random.shuffle(combined)
feature_test[:], labels_test[:] = zip(*combined)


feature = list(map(list,feature))
feature_test = list(map(list,feature_test))

prob = svm_problem(labels,feature)
param = svm_parameter('-t 2 -s 0')

print(len(feature))


m = svm_train(prob,param)

res = svm_predict(labels,feature,m)

res = svm_predict(labels_test,feature_test,m)

In [ ]:
svm_save_model("svm_v4_arial_28_digits_v2.model", m)

In [ ]:
model = svm_load_model('svm_v4_arial_28_digits_v1.model')

feature_test, labels_test = load_ocr_data(path_test)
feature_test = list(map(list,feature_test))
res = svm_predict(labels_test,feature_test,model)
print(res)

In [ ]:
model = svm_load_model('svm_v4_arial_28_digits_v2.model')
feature_test, labels_test = load_ocr_data(path_test,1)
import random
combined = list(zip(feature_test, labels_test))
random.shuffle(combined)
feature_test[:], labels_test[:] = zip(*combined)
feature_test = list(map(list,feature_test))
p_labels, p_acc, p_vals = svm_predict(labels_test,feature_test,model)
print(labels_test)
print(p_labels)
print(p_vals)
print(p_acc)

In [ ]:
print(res)